## Federated-Transfer learning Tutorial − Integrating Transfer learning to Federated Learning using MEDfl package

@Author : [MEDomics consortium](https://github.com/medomics/)

@Email : medomics.info@gmail.com

## Introduction

This notebook demonstrates the process of integrating [Transfer learning](https://ieeexplore.ieee.org/abstract/document/5288526/) using the *MEDfl* package. The primary objective of incorporating transfer learning with the package is to harness the capabilities of [Federated-Transfer learning](https://link.springer.com/chapter/10.1007/978-3-031-11748-0_3) in training models across different hospitals. In real-world scenarios, one of the clients or the aggregating server might possess a [pre-trained model](https://blogs.nvidia.com/blog/what-is-a-pretrained-ai-model/#:~:text=A%20pretrained%20AI%20model%20is,8%2C%202022%20by%20Angie%20Lee). Leveraging this pre-trained model offers advantages such as enhancing performance and reducing training time.

In some instances, a client may lack sufficient data to train a model entirely from scratch, hindering the ability to achieve optimal performance. Utilizing transfer learning becomes a viable strategy to maximize the benefits from each client, allowing the integration of previously learned knowledge to enhance model training and performance.

<img src="../Images/FTL_comp.png"  style="width:600px ;height:400px ; display:block ;margin:0 auto"> 

### EiCu Data 
This tutorial involves the utilization of the eICU dataset, a CSV file contains information on 200,860 patients, to train a binary classifier model.


## Tutorial Start

To integrate Transfer Learning into the *MEDfl* package, several sequential steps are essential:

1. **Importing a Pretrained Model:** Acquire or utilize a pretrained model containing learned knowledge from prior tasks or domains.
2. **Initialization at the Central Server:** The central server initiates its model by copying the pretrained model's parameters or weights.
3. **Initiating Federated Learning:** Upon initialization, the federated learning process begins, facilitating the exchange of model updates among participating clients for joint model training.

### Importing a Pretrained Model:
   - When importing a pretrained model, there are two options available:
   
     1. **Train and Save Locally:** Train a model localy, save it, and subsequently incorporate it for use.
     2. **External Source or Previous Work:** Import a pretrained model from an external source or retrieve it from a previous project.


### 1. Train and Save Localy
In this section, we aim to train a basic binary classifier model using the `eicu_sapsii_data.csv` dataset. Following the model training, we'll save the trained model for future utilization.

Imports

In [6]:
from MEDfl.LearningManager.utils import global_params

import sys
sys.path.append(global_params['base_url'])

import os
os.environ['PYTHONPATH'] = global_params['base_url']

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from MEDfl.LearningManager.model import Model
from MEDfl.LearningManager.utils import global_params

Read the data file

In [5]:
# Read the CSV file
data = pd.read_csv(global_params['base_url'] + '/notebooks/eicu_test.csv')

data.head()

,id,site_hospital,site_region,age,pao2fio2,uo,admissiontype,bicarbonate,bilirubin,bun,chron_dis,gcs,hr,potassium,sbp,sodium,tempc,wbc,event_death
0,stay147985,site73,Midwest,16,0.0,4,6,3,0,6,0,5,0,0,5,1,0,0,1
1,stay156248,site73,Midwest,7,0.0,0,6,0,0,0,0,0,0,0,5,0,0,0,0
2,stay156308,site60,Midwest,18,0.0,0,6,0,0,6,0,0,0,3,5,1,0,0,0
3,stay157820,site73,Midwest,12,0.0,11,6,3,0,10,0,0,0,0,0,1,0,0,0
4,stay159036,site73,Midwest,18,0.0,0,6,0,0,6,0,0,4,0,5,0,0,3,0


#### Data Preprocessing Steps

Here, we perform several essential data preprocessing steps:

1. **Drop Unnecessary Columns:** The code drops the 'subject_id' column and any other columns deemed unnecessary for the analysis. Additional columns can be added to the `columns_to_drop` list for removal.

2. **Define Features and Target Variable:** The features are defined by selecting all columns except the 'deceased' column, which is designated as the target variable for the binary classification task.

3. **Impute Missing Values:** Missing values in the selected features are imputed using the mean strategy. The `SimpleImputer` from the scikit-learn library is employed to fill missing values in the dataset.

4. **Preview the Transformed Data:** The `.head()` function is used to display the first few rows of the transformed dataset after preprocessing.

The code snippet provides a glimpse of the preprocessing steps, ensuring data cleanliness and preparation for training the binary classifier using the eICU dataset.


In [6]:
# Drop 'subject_id' column and any other unnecessary columns
columns_to_drop = ["id"]  # Add more columns to drop if needed
data.drop(columns=columns_to_drop, inplace=True)

# Define features and target variable
features = [col for col in data.columns if col != 'event_death']
target = 'event_death'

# Impute missing values using the mean strategy
imputer = SimpleImputer(strategy='mean')
encoder = LabelEncoder()
data["site_hospital"] = encoder.fit_transform(data["site_hospital"])
data["site_region"] = encoder.fit_transform(data["site_region"])

data[features] = imputer.fit_transform(data[features])

data.head()

,site_hospital,site_region,age,pao2fio2,uo,admissiontype,bicarbonate,bilirubin,bun,chron_dis,gcs,hr,potassium,sbp,sodium,tempc,wbc,event_death
0,67.0,0.0,16.0,0.0,4.0,6.0,3.0,0.0,6.0,0.0,5.0,0.0,0.0,5.0,1.0,0.0,0.0,1
1,67.0,0.0,7.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0
2,61.0,0.0,18.0,0.0,0.0,6.0,0.0,0.0,6.0,0.0,0.0,0.0,3.0,5.0,1.0,0.0,0.0,0
3,67.0,0.0,12.0,0.0,11.0,6.0,3.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
4,67.0,0.0,18.0,0.0,0.0,6.0,0.0,0.0,6.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,3.0,0


#### Modal initialisation

In [7]:


X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)


# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Define the neural network model using PyTorch
class BinaryClassifier(nn.Module):
    def __init__(self, input_dim):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 32)
        self.dropout2 = nn.Dropout(0.3)
        self.output = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.output(x)
        x = self.sigmoid(x)
        return x

# Initialize the model
input_dim = X_train_tensor.shape[1]
model = BinaryClassifier(input_dim)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Prepare data loaders for PyTorch
train_data = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)





In [10]:
input_dim

17

#### Modal Training 

In [8]:
# Train the model
epochs = 20
model.train()
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - Loss: {running_loss/len(train_loader)}")

Epoch 1/20 - Loss: 0.6682375013828278
Epoch 2/20 - Loss: 0.626903623342514
Epoch 3/20 - Loss: 0.5825505018234253
Epoch 4/20 - Loss: 0.5580604672431946
Epoch 5/20 - Loss: 0.5285914957523346
Epoch 6/20 - Loss: 0.49749096035957335
Epoch 7/20 - Loss: 0.48559616804122924
Epoch 8/20 - Loss: 0.5011755645275116
Epoch 9/20 - Loss: 0.47621373534202577
Epoch 10/20 - Loss: 0.4585097342729568
Epoch 11/20 - Loss: 0.48438641130924226
Epoch 12/20 - Loss: 0.47160635590553285
Epoch 13/20 - Loss: 0.4439723551273346
Epoch 14/20 - Loss: 0.44068617224693296
Epoch 15/20 - Loss: 0.4389739722013474
Epoch 16/20 - Loss: 0.4385830879211426
Epoch 17/20 - Loss: 0.4293429747223854
Epoch 18/20 - Loss: 0.43730728328227997
Epoch 19/20 - Loss: 0.42650032639503477
Epoch 20/20 - Loss: 0.4289227932691574


#### Evaluation

In [9]:
# Evaluation on test data
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    predictions = (outputs.squeeze() > 0.5).float()  # Convert probabilities to binary predictions

# Calculate metrics
accuracy = accuracy_score(y_test_tensor.numpy(), predictions.numpy())
print(f"Test Accuracy: {accuracy:.2f}")

# Classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_test_tensor.numpy(), predictions.numpy()))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test_tensor.numpy(), predictions.numpy()))

Test Accuracy: 0.88

Classification Report:
              precision    recall  f1-score   support

         0.0       0.88      1.00      0.93        70
         1.0       0.00      0.00      0.00        10

    accuracy                           0.88        80
   macro avg       0.44      0.50      0.47        80
weighted avg       0.77      0.88      0.82        80


Confusion Matrix:
[[70  0]
 [10  0]]


/home/local/USHERBROOKE/saho6810/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/local/USHERBROOKE/saho6810/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/local/USHERBROOKE/saho6810/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_

#### Save the Trained modal

To save the trained model for future use in the Learning Manager as a pre-trained model to activate Transfer Learning, we use the `save_model` method available in the `Model` class within the *MEDfl* package. This method requires two parameters: the `model` to be saved and the `model_name`.

In [10]:
# Save the trained model 
Model.save_model(model=model , model_name='binary_classifier')

#### Import the Pre-trained Modal

In [11]:
# Load the saved Model 
model = Model.load_model(model_name='binary_classifier')

# Ensure the model is in evaluation mode for inference
model.eval()

BinaryClassifier(
  (fc1): Linear(in_features=17, out_features=64, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (dropout2): Dropout(p=0.3, inplace=False)
  (output): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

### Modifying the Pretrained Model

Following the initial training of the model, several modifications can be applied to adapt the pretrained model to the new context of the federated learning problem. Various options for modifying the pretrained model include:

1. **Fine-Tuning Layers:** Selective updating of specific layers allows fine-tuning the model. Lower-level layers may retain learned representations, while higher-level layers adapt to the new task or domain.

2. **Adjusting Learning Rates:** Modification of learning rates for different layers or layer groups aids in emphasizing specific layers' importance or controlling updates to pretrained layers.

3. **Changing Activation Functions:** Experimentation with different activation functions in certain layers influences the model's learning behavior and adaptability to new data.

4. **Transfer Learning Strategies:** Employing strategies like feature extraction or full retraining based on available data and computational resources to leverage pretrained model knowledge.

5. **Customized Loss Functions:** Designing or utilizing domain-specific loss functions tailored to address specific requirements of the federated learning problem.

These modification options provide flexibility and customization in adapting the pretrained model, ensuring its alignment with the unique demands and characteristics of the federated learning or transfer learning tasks.


1. Fine-Tuning Layers

In [12]:
# For example, fine-tune the last layers of the pretrained model
for param in model.parameters():
    param.requires_grad = False  # Freeze all layers

for param in model.output.parameters():
    param.requires_grad = True  # Unfreeze output layer for fine-tuning


2. Adjusting Learning Layers 

In [13]:
# Define different learning rates for different layer groups
optimizer = optim.Adam([
    {'params': model.fc1.parameters(), 'lr': 0.0001},  # Lower LR for certain layers
    {'params': model.fc2.parameters(), 'lr': 0.0005},  # Higher LR for certain layers
    {'params': model.output.parameters(), 'lr': 0.001}  # LR for output layer
])

3. Changing Activation functions 

In [14]:
# Experiment with different activation functions in certain layers
# For instance, using LeakyReLU in place of ReLU
model.fc1 = nn.Linear(input_dim, 64)
model.fc2 = nn.Linear(64, 32)
model.output = nn.Linear(32, 1)
model.leaky_relu = nn.LeakyReLU()  # Introduce LeakyReLU activation function
model.sigmoid = nn.Sigmoid()

In [15]:
# Save the Updated Modal 
Model.save_model(model=model , model_name="updated_model")

In [16]:
# Load the updated modal 
updated_model = Model.load_model(model_name="updated_model")

updated_model

BinaryClassifier(
  (fc1): Linear(in_features=17, out_features=64, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (dropout2): Dropout(p=0.3, inplace=False)
  (output): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (leaky_relu): LeakyReLU(negative_slope=0.01)
)

### Finally
After having the pretrained modal ready to use we have to pass it the Server class of the `MEDfl` package so the federated learing process can start 